In [11]:
import pandas as pd
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
import numpy as np

In [32]:
p9 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2009.csv', sep=',', low_memory=False)

In [20]:
p10 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2010.csv', sep=',', low_memory=False)

In [24]:
p11 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2011.csv', sep=',', low_memory=False)

In [25]:
p12 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2012.csv', sep=',', low_memory=False)

In [26]:
p13 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2013.csv', sep=',', low_memory=False)

In [27]:
p14 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2014.csv', sep=',', low_memory=False)

In [29]:
p15 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2015.csv', sep=',', low_memory=False)

In [30]:
p16 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2016.csv', sep=',', low_memory=False)

In [31]:
p17 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2017.csv', sep=',', low_memory=False)

In [34]:
p18 = pd.read_csv(r'nflscrapR-data/play_by_play_data/regular_season/reg_pbp_2018.csv', sep=',', low_memory=False)

In [42]:
passergroup18 = p18.groupby(['posteam', 'passer_player_name'])["pass_touchdown", "yards_gained"].apply(lambda x : x.astype(int).sum())
passergroup18.head()

pass_touchdown  yards_gained
posteam passer_player_name                              
ARI     J.Rosen                         11          1960
        L.Fitzgerald                     1            32
        M.Glennon                        1           166
        S.Bradford                       2           367
ATL     M.Ryan                          35          4632